In [1]:
%defaultDatasource jdbc:h2:mem:db4

# Modelo Conceitual.

<img src="conceitual.jpeg">


# Modelo Lógico.



Paciente(_id_, idade, sexo, eh_fumante [B], hist_diabetes[B], hist_doenca_arterial[B])
Primary Key: id

Sintoma(_id_paciente_, local_dor[str], provocada_por_esforço[B], melhorou_com_descanso[B], tipo_dor_toracica[str])
Primary Key: id_paciente
Foreigner Key: id_paciente

ECG(_id_paciente_, resultado, protocolo_exercicio, duracao_exercicio, batimento_max, pico_pressao)
Primary Key: id_paciente
Foreigner Key: id_paciente

Triagem(_id_paciente_, pressao_repouso, colesterol, glicose)
Primary Key: id_paciente
Foreigner Key: id_paciente

Diagnostico(_id_paciente_, status_angiografico, vaso_atingido)
Primary Key: id_paciente
Foreigner Key: id_paciente

In [2]:
--Task List:


--Paciente(_id_, idade, sexo, eh_fumante [B], hist_diabetes[B], hist_doenca_arterial[B])
--Primary Key: id


DROP TABLE IF EXISTS Paciente;
DROP TABLE IF EXISTS Sintoma;
DROP TABLE IF EXISTS ECG;
DROP TABLE IF EXISTS Triagem;
DROP TABLE IF EXISTS Diagnostico;

CREATE TABLE Paciente (
id VARCHAR(36),
idade FLOAT,
genero FLOAT, 
PRIMARY KEY(id)
) AS SELECT
    uid,
    idade,
    genero
FROM CSVREAD('cleveland_id.csv');


--Sintoma(_id_paciente_, local_dor[str], provocada_por_esforço[B], melhorou_com_descanso[B], tipo_dor_toracica[str])
--Primary Key: id_paciente
--Foreigner Key: id_paciente

CREATE TABLE Sintoma (
id_paciente VARCHAR(36),
tipo_dor_toracica FLOAT,
-- local_dor VARCHAR(100),
-- provocada_por_esforço INTEGER,   
-- melhorou_com_descanso VARCHAR(100),
-- tipo_dor_toracica VARCHAR(100),  
PRIMARY KEY(id_paciente),
FOREIGN KEY(id_paciente)
    REFERENCES Paciente(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION    
) AS SELECT
    uid,
    tipo_dor_toracica
FROM CSVREAD('cleveland_id.csv');


--ECG(_id_paciente_, resultado, protocolo_exercicio, duracao_exercicio, batimento_max, pico_pressao)
--Primary Key: id_paciente
--Foreigner Key: id_paciente
    
CREATE TABLE ECG (
id_paciente VARCHAR(36),
resultado FLOAT,
batimento_max FLOAT,
induziu_angina FLOAT,
depressao_st FLOAT,
declive_st FLOAT,
numero_vasos FLOAT,
thal FLOAT,
PRIMARY KEY(id_paciente),
FOREIGN KEY(id_paciente)
    REFERENCES Paciente(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION  

) AS SELECT
    uid,
    resultado,
    batimento_max,
    induziu_angina,
    depressao_st,
    declive_st,
    numero_vasos,
    thal
FROM CSVREAD('cleveland_id.csv');


--Triagem(_id_paciente_, pressao_repouso, colesterol, glicose)
--Primary Key: id_paciente
--Foreigner Key: id_paciente


CREATE TABLE Triagem (
id_paciente VARCHAR(36),
pressao_repouso FLOAT,
colesterol FLOAT,
glicose FLOAT,
PRIMARY KEY(id_paciente),
FOREIGN KEY(id_paciente)
REFERENCES Paciente(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION  

) AS SELECT
    uid,
    pressao_repouso,
    colesterol,
    glicose
FROM CSVREAD('cleveland_id.csv');

--Diagnostico(_id_paciente_, status_angiografico, vaso_atingido)
--Primary Key: id_paciente
--Foreigner Key: id_paciente


CREATE TABLE Diagnostico (
id_paciente VARCHAR(36),
diagnostico FLOAT,
PRIMARY KEY(id_paciente),
FOREIGN KEY(id_paciente)
REFERENCES Paciente(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION  

) AS SELECT
    uid,
    diagnostico
FROM CSVREAD('cleveland_id.csv');


 - Queries descritivas sobre as principais tabelas.
     
     
    
  Tais queries tem como objetivo melhorar a compreensão dos dados por um possível usuário leigo.
  

In [3]:
SELECT * FROM Paciente;

---

In [4]:
SELECT * FROM Sintoma;

---

In [5]:
SELECT * FROM ECG;

---

#### Dentre  uma escala de 0 a 4 a partir do atributo "diagnostico" temos : ####

    0 : Saudável
    1 à 4 : Não saudável 
    
    
    
    As proximas duas Queries indicarão o numero de paciêntes saudáveis e não saudáveis respectivamente.

In [6]:
SELECT COUNT(*) FROM Paciente P, Diagnostico D WHERE D.diagnostico = 0 AND D.id_paciente = P.id;

164

---

In [7]:
SELECT COUNT(*) FROM Paciente P, Diagnostico D WHERE D.diagnostico > 0 AND D.id_paciente = P.id;

139

---

 Utilizamos a função Round para arredondar o numero, com a consulta sendo o primeiro parametro passado para a função.

A seguinte query calculará a média das pressões arteriais de paciêntes em repouso.


In [8]:
select round(AVG(Triagem.pressao_repouso),2) from Triagem

131.69

---

 Essa Query relaciona o numero de pessoas que estão em quadro não saudável e que tem a pressão em repouso maior que a média.

In [9]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, Triagem T
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (T.id_paciente = P.id AND T.pressao_repouso > (SELECT AVG(Triagem.pressao_repouso) FROM Triagem));

68

---

Análise em função da pressão de repouso agora acima de 120.0

In [10]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, Triagem T
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (T.id_paciente = P.id AND T.pressao_repouso > 120.0);

102

---
Análise da média dos resultados do exame de colesterol.

In [11]:
select round(AVG(Triagem.colesterol),2) from Triagem


246.69

---

 Essa Query relaciona o numero de pessoas que estão em quadro não saudável e que tem o resultado do exame de colesterol acima da média.
 

In [12]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, Triagem T
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (T.id_paciente = P.id AND T.colesterol > (SELECT AVG(Triagem.colesterol) FROM Triagem));

74

---

Análise em função do resultado do exame de colesterol agora acima de 190.0

In [13]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, Triagem T
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (T.id_paciente = P.id AND T.colesterol > 190.0);

123

---


 Essa Query relaciona o numero de pessoas que estão em quadro saudável e não saudavel e que tem o resultado de glicose = 1
 

In [26]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, Triagem T
WHERE (D.diagnostico = 0 AND D.id_paciente = P.id) AND (T.id_paciente = P.id AND T.glicose = 1);

SELECT COUNT(*)
FROM Diagnostico D, Paciente P, Triagem T
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (T.id_paciente = P.id AND T.glicose = 1);

---


 Essa Query relaciona o numero de pessoas que estão em quadro saudável e não saudavel e que tem o resultado de glicose = 0
 

In [27]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, Triagem T
WHERE (D.diagnostico = 0 AND D.id_paciente = P.id) AND (T.id_paciente = P.id AND T.glicose = 0);

SELECT COUNT(*)
FROM Diagnostico D, Paciente P, Triagem T
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (T.id_paciente = P.id AND T.glicose = 0);

---

Análise da média dos resultados do batimento máximo obtido no exame ECG.

In [16]:
select round(AVG(batimento_max),2) from ECG;

149.61

---


Essa Query relaciona o numero de pessoas que estão em quadro não saudável e que tem o resultado do batimento cardiaco máximo acima da média.

In [17]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.batimento_max > (SELECT AVG(batimento_max) FROM ECG));

50

---

Essa Query relaciona o numero de pessoas que estão em quadro não saudável e que tem o resultado do exame de batimento cardiaco máximo acima de 170bpm.



In [18]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico = 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.batimento_max > 170);

48

---

Essa Query relaciona o numero de pessoas que estão em quadro não saudável e que tem o resultado do exame de ECG em repouso sendo possivel como resultado 0 , 1 e 2.

A descrição do estado "2" é descrita abaixo.
"showing probable or definite left ventricular hypertrophy by Estes' criteria "

In [19]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.resultado = 2);

80

---

Análise da média dos resultados de exames onde há depressões em exames TP durante o ECG.

In [20]:

select round(AVG(depressao_st),2) from ECG;

1.04

---

Essa Query relaciona o numero de pessoas que estão em quadro saudável e o restulado onde há depressões **abaixo da média** no exame durante o ECG.

In [21]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico = 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.depressao_st < (SELECT AVG(depressao_st) FROM ECG));

123

---

Essa Query relaciona o numero de pessoas que estão em quadro não saudável e o restulado onde há depressões **acima da média** no exame durante o ECG.

In [22]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.depressao_st > (SELECT AVG(depressao_st) FROM ECG));

82

---


Essa Query relaciona o numero de pessoas que estão em quadro não saudável e o restulado onde há depressões **acima de 3** no exame durante o ECG.

In [25]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico = 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.depressao_st > 3);

SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.depressao_st > 3);

---
Essa Query relaciona o numero de pessoas que estão em quadro não saudável e o restulado onde há depressões **abaixo de 0.6** no exame durante o ECG.

In [24]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico = 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.depressao_st < 0.6);

SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.depressao_st < 0.6);



---
Essa Query relaciona o numero de pessoas que estão em quadro saudavel e não saudável e o restulado onde não há vasos coloridos pela Fluoroscopia.

In [28]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico = 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.numero_vasos = 0);

SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.numero_vasos = 0);



---
Essa Query relaciona o numero de pessoas que estão em quadro saudavel e não saudável e o restulado onde há mais de 2 vasos coloridos pela Fluoroscopia.

In [29]:
SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico = 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.numero_vasos >= 2);

SELECT COUNT(*)
FROM Diagnostico D, Paciente P, ECG E
WHERE (D.diagnostico > 0 AND D.id_paciente = P.id) AND (E.id_paciente = P.id AND E.numero_vasos >= 2);

